# Task 1: Exploratory Data Analysis and Data Preprocessing

This notebook performs EDA on the CFPB complaint dataset and prepares it for the RAG pipeline.

## Objectives:
1. Load the full CFPB complaint dataset
2. Perform initial EDA
3. Filter dataset to specified products
4. Clean text narratives
5. Save cleaned dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

## 1. Load the Dataset

In [ ]:
# Load the dataset
data_path = Path('../data/complaints.csv')

if not data_path.exists():
    print("Dataset not found. Please run src/download_data.py first.")
    print("Or download manually from: https://www.consumerfinance.gov/data-research/consumer-complaints/")
else:
    print("Loading dataset...")
    df = pd.read_csv(data_path, low_memory=False)
    print(f"Dataset loaded successfully!")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Basic info
df.info()

## 2. Exploratory Data Analysis

### 2.1 Distribution of Complaints Across Products

In [ ]:
# Get product distribution
product_counts = df['Product'].value_counts()
print("Top 20 Products by Complaint Count:")
print(product_counts.head(20))

# Visualize top 20 products
plt.figure(figsize=(12, 8))
product_counts.head(20).plot(kind='barh')
plt.xlabel('Number of Complaints')
plt.ylabel('Product')
plt.title('Top 20 Products by Complaint Count')
plt.tight_layout()
plt.show()

### 2.2 Analysis of Consumer Complaint Narratives

In [ ]:
# Count complaints with and without narratives
total_complaints = len(df)
complaints_with_narrative = df['Consumer complaint narrative'].notna().sum()
complaints_without_narrative = df['Consumer complaint narrative'].isna().sum()

print(f"Total Complaints: {total_complaints:,}")
print(f"Complaints WITH narratives: {complaints_with_narrative:,} ({complaints_with_narrative/total_complaints*100:.2f}%)")
print(f"Complaints WITHOUT narratives: {complaints_without_narrative:,} ({complaints_without_narrative/total_complaints*100:.2f}%)")

# Visualize
plt.figure(figsize=(8, 6))
labels = ['With Narrative', 'Without Narrative']
sizes = [complaints_with_narrative, complaints_without_narrative]
colors = ['#66b3ff', '#ff9999']
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Complaints With vs Without Narratives')
plt.axis('equal')
plt.show()

In [ ]:
# Calculate word count for narratives
def count_words(text):
    """Count words in a text string"""
    if pd.isna(text):
        return 0
    return len(str(text).split())

print("Calculating word counts for narratives...")
df['narrative_word_count'] = df['Consumer complaint narrative'].apply(count_words)

# Filter to only complaints with narratives for analysis
df_with_narrative = df[df['narrative_word_count'] > 0].copy()

print(f"\nNarrative Word Count Statistics:")
print(df_with_narrative['narrative_word_count'].describe())

In [ ]:
# Visualize word count distribution
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Histogram
axes[0].hist(df_with_narrative['narrative_word_count'], bins=100, edgecolor='black')
axes[0].set_xlabel('Word Count')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Narrative Word Counts')
axes[0].axvline(df_with_narrative['narrative_word_count'].median(), 
                color='red', linestyle='--', label=f"Median: {df_with_narrative['narrative_word_count'].median():.0f}")
axes[0].legend()

# Boxplot
axes[1].boxplot(df_with_narrative['narrative_word_count'], vert=False)
axes[1].set_xlabel('Word Count')
axes[1].set_title('Boxplot of Narrative Word Counts')

plt.tight_layout()
plt.show()

In [ ]:
# Identify very short and very long narratives
very_short_threshold = 10  # words
very_long_threshold = df_with_narrative['narrative_word_count'].quantile(0.95)

very_short = df_with_narrative[df_with_narrative['narrative_word_count'] < very_short_threshold]
very_long = df_with_narrative[df_with_narrative['narrative_word_count'] > very_long_threshold]

print(f"Very short narratives (< {very_short_threshold} words): {len(very_short):,} ({len(very_short)/len(df_with_narrative)*100:.2f}%)")
print(f"Very long narratives (> {very_long_threshold:.0f} words): {len(very_long):,} ({len(very_long)/len(df_with_narrative)*100:.2f}%)")
print(f"\nExample of a very short narrative:")
if len(very_short) > 0:
    print(very_short.iloc[0]['Consumer complaint narrative'])
print(f"\nExample of a very long narrative (first 500 chars):")
if len(very_long) > 0:
    print(str(very_long.iloc[0]['Consumer complaint narrative'])[:500] + "...")

## 3. Data Filtering

In [ ]:
# Define target products (Note: checking exact product names in dataset)
# The problem mentions: Credit card, Personal loan, Savings account, Money transfers
# We need to check actual product names in the dataset

print("All unique products in dataset:")
all_products = df['Product'].unique()
for i, product in enumerate(sorted(all_products), 1):
    print(f"{i}. {product}")

In [ ]:
# Map to the specified products
# Looking for: Credit card, Personal loan, Savings account, Money transfers
target_products = [
    'Credit card or prepaid card',
    'Credit card',
    'Prepaid card',
    'Payday loan, title loan, or personal loan',
    'Personal loan',
    'Checking or savings account',
    'Money transfer, virtual currency, or money service',
    'Money transfers'
]

# Filter based on available products - let's be flexible
# First, let's see what matches
matching_products = [p for p in all_products if any(keyword in p.lower() for keyword in 
                     ['credit card', 'personal loan', 'payday loan', 'savings', 'checking', 'money transfer'])]

print("\nMatching products found:")
for product in matching_products:
    count = len(df[df['Product'] == product])
    print(f"  {product}: {count:,} complaints")

In [ ]:
# Filter dataset
print("Filtering dataset...")
print(f"Original dataset size: {len(df):,}")

# Step 1: Filter by products
df_filtered = df[df['Product'].isin(matching_products)].copy()
print(f"After product filtering: {len(df_filtered):,}")

# Step 2: Remove records with empty narratives
df_filtered = df_filtered[df_filtered['Consumer complaint narrative'].notna()].copy()
df_filtered = df_filtered[df_filtered['Consumer complaint narrative'].astype(str).str.strip() != ''].copy()
print(f"After removing empty narratives: {len(df_filtered):,}")

print(f"\nFiltered dataset shape: {df_filtered.shape}")

In [ ]:
# Visualize filtered product distribution
plt.figure(figsize=(10, 6))
df_filtered['Product'].value_counts().plot(kind='barh')
plt.xlabel('Number of Complaints')
plt.ylabel('Product')
plt.title('Filtered Dataset: Product Distribution')
plt.tight_layout()
plt.show()

## 4. Text Cleaning

In [ ]:
def clean_text(text):
    """
    Clean complaint narrative text:
    - Convert to lowercase
    - Remove special characters
    - Remove boilerplate text
    - Normalize whitespace
    """
    if pd.isna(text):
        return ""
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove common boilerplate phrases
    boilerplate_phrases = [
        r'i am writing to file a complaint\s*',
        r'dear sir or madam\s*',
        r'to whom it may concern\s*',
        r'i am writing to\s*',
        r'xx+',  # Remove sequences of X's (often used for redaction)
    ]
    
    for phrase in boilerplate_phrases:
        text = re.sub(phrase, '', text, flags=re.IGNORECASE)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove special characters but keep basic punctuation
    text = re.sub(r'[^a-z0-9\s.,!?\-]', ' ', text)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

# Test the cleaning function
sample_text = df_filtered.iloc[0]['Consumer complaint narrative']
print("Original text (first 300 chars):")
print(sample_text[:300])
print("\nCleaned text (first 300 chars):")
print(clean_text(sample_text)[:300])

In [ ]:
# Apply cleaning to all narratives
print("Cleaning narratives...")
df_filtered['cleaned_narrative'] = df_filtered['Consumer complaint narrative'].apply(clean_text)

# Remove any that became empty after cleaning
df_filtered = df_filtered[df_filtered['cleaned_narrative'].str.len() > 0].copy()
print(f"After cleaning and removing empty: {len(df_filtered):,}")

# Recalculate word count for cleaned narratives
df_filtered['cleaned_word_count'] = df_filtered['cleaned_narrative'].apply(count_words)

print("\nCleaned Narrative Statistics:")
print(df_filtered['cleaned_word_count'].describe())

## 5. Save Cleaned Dataset

In [ ]:
# Select relevant columns for final dataset
columns_to_save = [
    'Complaint ID',
    'Product',
    'Sub-product',
    'Issue',
    'Sub-issue',
    'Consumer complaint narrative',
    'cleaned_narrative',
    'Company',
    'State',
    'Date received',
    'Date sent to company'
]

# Filter to only columns that exist
available_columns = [col for col in columns_to_save if col in df_filtered.columns]
df_final = df_filtered[available_columns].copy()

# Save to CSV
output_path = Path('../data/filtered_complaints.csv')
df_final.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")
print(f"Final dataset shape: {df_final.shape}")
print(f"\nColumns saved: {list(df_final.columns)}")

## 6. Summary Statistics

In [ ]:
print("="*60)
print("FINAL DATASET SUMMARY")
print("="*60)
print(f"Total complaints in cleaned dataset: {len(df_final):,}")
print(f"\nProduct distribution:")
print(df_final['Product'].value_counts())
print(f"\nNarrative length statistics (cleaned):")
print(df_filtered['cleaned_word_count'].describe())
print("="*60)

## EDA Summary

### Key Findings:

1. **Dataset Overview**: The CFPB complaint dataset contains a large number of consumer complaints across various financial products. A significant portion of complaints include detailed narratives describing the issues.

2. **Narrative Analysis**: 
   - The dataset shows considerable variation in narrative length, with some complaints being very brief (under 10 words) and others being quite detailed (over several hundred words).
   - The median narrative length provides a good indicator of typical complaint detail level.
   - Not all complaints include narratives, which is why filtering for non-empty narratives is crucial for our RAG pipeline.

3. **Product Focus**: By filtering to the specified product categories (credit cards, personal loans, savings accounts, and money transfers), we've created a focused dataset that aligns with the project requirements. The filtered dataset maintains a good distribution across these product categories, ensuring our RAG system will have diverse examples to work with.

4. **Text Cleaning**: The cleaning process successfully normalized the text by:
   - Converting all text to lowercase for consistency
   - Removing boilerplate phrases that don't add semantic value
   - Eliminating special characters and URLs that could interfere with embedding quality
   - Preserving the core complaint content while improving text quality for downstream processing

The cleaned dataset is now ready for text chunking, embedding, and vector store indexing in Task 2.